# Best Practices for Using Runners

In [1]:
%pip uninstall -y todd_ai
%pip install --no-build-isolation --extra-index-url https://pypi.org/simple .. > /dev/null


Found existing installation: todd-ai 0.4.0
Uninstalling todd-ai-0.4.0:
  Successfully uninstalled todd-ai-0.4.0
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pathlib
import tempfile
import time
from pprint import pprint
from typing import Any, NoReturn, TypedDict

import torch
import torch.nn.functional as F
import torch.utils.data
from torch import nn

import todd
from todd.runners import Memo

[2024-02-07 15:04:10,052 56800:140704380690048][patches.py:9 todd <module>] INFO: `ipdb` is installed. Using it for debugging.
/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


## Preparation

### Models

In [3]:
@todd.ModelRegistry.register_()
class RunnerModel(nn.Module):

    def __init__(self) -> None:
        super().__init__()
        self._weight = torch.nn.Parameter(torch.tensor(0.0))

    @property
    def weight(self) -> torch.nn.Parameter:
        return self._weight

    def _forward(self, x: torch.Tensor) -> torch.Tensor:
        return x * self._weight

    def forward(
        self,
        runner: todd.runners.BaseRunner,
        batch,
        memo: Memo,
        *args,
        **kwargs,
    ) -> Memo:
        log: dict[str, Any] | None = memo.get("log")
        y = self._forward(batch["x"])
        loss = F.l1_loss(y, batch["y"])
        memo["loss"] = loss
        if log is not None:
            log["batch"] = str(batch)
            log["weight"] = f"{self._weight.item():.3f}"
            log["loss"] = f"{loss:.3f}"
        return memo

### Datasets

In [4]:
class Sample(TypedDict):
    x: int
    y: int

In [5]:
@todd.DatasetRegistry.register_()
class RunnerDataset(torch.utils.data.Dataset[int]):

    def __init__(self, n: int) -> None:
        self._data = list(range(1, n + 1))

    def __len__(self) -> int:
        return len(self._data)

    def __getitem__(self, index: int) -> Sample:
        x = self._data[index]
        return Sample(x=x, y=x * 2)

In [6]:
class Batch(TypedDict):
    x: torch.Tensor
    y: torch.Tensor

## Validators

In [7]:
config = todd.Config(
    type='Validator',
    name='validator',
    dataloader=dict(batch_size=1, dataset=dict(type='RunnerDataset', n=20)),
    strategy=dict(type='BaseStrategy', model=dict(type='RunnerModel')),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree $work_dirs


[2024-02-07 15:04:12,282 56800:140704380690048][base.py:55 todd.Validator.validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpivghh32q
└── validator

2 directories, 0 files


In [8]:
config = todd.Config(
    type='Validator',
    name='validator',
    dataloader=dict(batch_size=1, dataset=dict(type='RunnerDataset', n=20)),
    strategy=dict(type='BaseStrategy', model=dict(type='RunnerModel')),
    callbacks=[dict(type='LogCallback', interval=5)],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree $work_dirs


[2024-02-07 15:04:12,628 56800:140704380690048][base.py:55 todd.Validator.validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-07 15:04:12,633 56800:140704380690048][log.py:91 todd.Validator.validator after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-02-07 15:04:12,637 56800:140704380690048][log.py:91 todd.Validator.validator after_run_iter] INFO: Iter [10/20] batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-02-07 15:04:12,640 56800:140704380690048][log.py:91 todd.Validator.validator after_run_iter] INFO: Iter [15/20] batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-02-07 15:04:12,644 56800:140704380690048][log.py:91 todd.Validator.validator after_run_iter] INFO: Iter [20/20] batch={'x': tensor([20]), 'y': tensor([40])} weight=0.000 loss=40.000



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpkoyphpsp
└── validator

2 directories, 0 files


## Trainers

### Iteration Based

In [9]:
config = todd.Config(
    type="IterBasedTrainer",
    name="iter_based_trainer",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[dict(type="LogCallback", interval=1)],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-07 15:04:12,960 56800:140704380690048][base.py:55 todd.IterBasedTrainer.iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-07 15:04:12,965 56800:140704380690048][log.py:91 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [1/8] batch={'x': tensor([ 4, 10]), 'y': tensor([ 8, 20])} weight=0.000 loss=14.000
[2024-02-07 15:04:12,967 56800:140704380690048][log.py:91 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [2/8] batch={'x': tensor([6, 1]), 'y': tensor([12,  2])} weight=0.000 loss=7.000
[2024-02-07 15:04:12,969 56800:140704380690048][log.py:91 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [3/8] batch={'x': tensor([2, 3]), 'y': tensor([4, 6])} weight=0.000 loss=5.000
[2024-02-07 15:04:12,971 56800:140704380690048][log.py:91 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [4/8] batch={'x': tensor([9, 8]), 'y': tensor([18, 16])} weight=0.000 loss=17.000
[2024-02

### Epoch Based

In [10]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="epoch_based_trainer",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[dict(type="LogCallback", interval=1)],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-07 15:04:12,996 56800:140704380690048][base.py:55 todd.EpochBasedTrainer.epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-07 15:04:12,998 56800:140704380690048][log.py:97 todd.EpochBasedTrainer.epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2024-02-07 15:04:13,001 56800:140704380690048][log.py:91 todd.EpochBasedTrainer.epoch_based_trainer after_run_iter] INFO: Iter [1/15] batch={'x': tensor([3, 6]), 'y': tensor([ 6, 12])} weight=0.000 loss=9.000
[2024-02-07 15:04:13,003 56800:140704380690048][log.py:91 todd.EpochBasedTrainer.epoch_based_trainer after_run_iter] INFO: Iter [2/15] batch={'x': tensor([5, 8]), 'y': tensor([10, 16])} weight=0.000 loss=13.000
[2024-02-07 15:04:13,005 56800:140704380690048][log.py:91 todd.EpochBasedTrainer.epoch_based_trainer after_run_iter] INFO: Iter [3/15] batch={'x': tensor([1, 2]), 'y': tensor([2, 4])} weight=0.000 loss=3.000
[2024-02-07 15:04:13,007 56800:140704380690048][log.py:91 todd.EpochBa

## Callbacks

### Log

In [11]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    dataloader=dict(batch_size=1, dataset=dict(type="RunnerDataset", n=20)),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            collect_env=dict(verbose=False),
        ),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-07 15:04:13,185 56800:140704380690048][log.py:53 todd.Validator.log_callback init] INFO: 
platform: macOS-14.0
nvidia_smi: None
python_version: 3.11.7 (main, Dec  4 2023, 18:10:11) [Clang 15.0.0 (clang-1500.1.0.2.5)]
pytorch_version: 2.0.1
torchvision_version: 0.15.2
opencv_version: 4.7.0
todd_version: 0.4.0
cuda_home: None
git_commit_id: f453075
git_status: 
M  pyproject.toml
M  todd/base/__init__.py
R  todd/utils/envs.py -> todd/base/envs.py
M  todd/base/registries.py
M  todd/runners/callbacks/__init__.py
A  todd/runners/callbacks/git.py
M  todd/runners/callbacks/log.py
M  todd/runners/types.py
M  todd/utils/__init__.py
M  todd/utils/generic_tensors.py
MM tutorials/runners.ipynb
[2024-02-07 15:04:13,186 56800:140704380690048][base.py:55 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-07 15:04:13,192 56800:140704380690048][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': te

In [12]:
config = todd.Config(
    type='Validator',
    name='log_callback',
    dataloader=dict(batch_size=1, dataset=dict(type='RunnerDataset', n=20)),
    strategy=dict(type='BaseStrategy', model=dict(type='RunnerModel')),
    callbacks=[
        dict(
            type='LogCallback',
            interval=5,
            with_file_handler=True,
        ),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo
    !cat {work_dirs}/log_callback/*.log


[2024-02-07 15:04:13,223 56800:140704380690048][base.py:55 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-07 15:04:13,227 56800:140704380690048][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-02-07 15:04:13,229 56800:140704380690048][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-02-07 15:04:13,232 56800:140704380690048][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-02-07 15:04:13,235 56800:140704380690048][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [20/20] batch={'x': tensor([20]), 'y': tensor([40])} weight=0.000 loss=40.000



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpjurzi5ju
└── log_callback
    └── 2024-02-07T15-04-13_223241-08-00.log

2 directories, 1 file

[2024-02-07 15:04:13,223 56800:140704380690048][base.py:55 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-07 15:04:13,227 56800:140704380690048][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-02-07 15:04:13,229 56800:140704380690048][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-02-07 15:04:13,232 56800:140704380690048][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-02-07 15:04:13,235 56800:140704380690048][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [20/20] batch={'x':

In [13]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    dataloader=dict(batch_size=1, dataset=dict(type="RunnerDataset", n=20)),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            eta=dict(type="AverageETA"),
        ),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.strategy.module.register_forward_hook(
        lambda *args, **kwargs: time.sleep(0.1)
    )
    runner.run()

[2024-02-07 15:04:13,829 56800:140704380690048][base.py:55 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-07 15:04:14,348 56800:140704380690048][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] ETA 0:00:01 batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-02-07 15:04:14,864 56800:140704380690048][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] ETA 0:00:01 batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-02-07 15:04:15,386 56800:140704380690048][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] ETA 0:00:00 batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-02-07 15:04:15,910 56800:140704380690048][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [20/20] ETA 0:00:00 batch={'x': tensor([20]), 'y': tensor([40])} weight=0.000 loss=40.000


In [14]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    dataloader=dict(batch_size=1, dataset=dict(type="RunnerDataset", n=20)),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            eta=dict(type="EMA_ETA", decay=0.2),
        ),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.strategy.module.register_forward_hook(
        lambda *args, **kwargs: time.sleep(0.1 * min(10, runner.iter_))
    )
    runner.run()

[2024-02-07 15:04:15,926 56800:140704380690048][base.py:55 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-07 15:04:17,451 56800:140704380690048][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] ETA 0:00:03 batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-02-07 15:04:21,469 56800:140704380690048][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] ETA 0:00:04 batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-02-07 15:04:26,497 56800:140704380690048][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] ETA 0:00:03 batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-02-07 15:04:31,517 56800:140704380690048][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [20/20] ETA 0:00:00 batch={'x': tensor([20]), 'y': tensor([40])} weight=0.000 loss=40.000


In [15]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    dataloader=dict(batch_size=1, dataset=dict(type="RunnerDataset", n=20)),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            collect_env=dict(verbose=False),
            with_file_handler=True,
            eta=dict(type="AverageETA"),
        ),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-07 15:04:31,658 56800:140704380690048][log.py:53 todd.Validator.log_callback init] INFO: 
platform: macOS-14.0
nvidia_smi: None
python_version: 3.11.7 (main, Dec  4 2023, 18:10:11) [Clang 15.0.0 (clang-1500.1.0.2.5)]
pytorch_version: 2.0.1
torchvision_version: 0.15.2
opencv_version: 4.7.0
todd_version: 0.4.0
cuda_home: None
git_commit_id: f453075
git_status: 
M  pyproject.toml
M  todd/base/__init__.py
R  todd/utils/envs.py -> todd/base/envs.py
M  todd/base/registries.py
M  todd/runners/callbacks/__init__.py
A  todd/runners/callbacks/git.py
M  todd/runners/callbacks/log.py
M  todd/runners/types.py
M  todd/utils/__init__.py
M  todd/utils/generic_tensors.py
MM tutorials/runners.ipynb
[2024-02-07 15:04:31,660 56800:140704380690048][base.py:55 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-07 15:04:31,665 56800:140704380690048][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] ETA 0:00:00 batch={'x': tensor([

### Git

In [16]:
config = todd.Config(
    type="Validator",
    name="git_callback",
    dataloader=dict(batch_size=1, dataset=dict(type="RunnerDataset", n=20)),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type="GitCallback", diff='HEAD -- ":(exclude)*.ipynb"'),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )

    !echo
    !cat {work_dirs}/git_callback/*.log

[2024-02-07 15:04:31,746 56800:140704380690048][git.py:50 todd.Validator.git_callback init] INFO: Saving git diff to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpky6migyw/git_callback/git_diff_2024-02-07T15-04-31_746637-08-00.log
[2024-02-07 15:04:31,749 56800:140704380690048][base.py:55 todd.Validator.git_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R



diff --git a/pyproject.toml b/pyproject.toml
index 83382b7..aa38442 100644
--- a/pyproject.toml
+++ b/pyproject.toml
@@ -78,6 +78,7 @@ test = [
 
 [project.scripts]
 configs_diff = 'todd.base.configs:diff_cli'
+collect_env = 'todd.base.envs:collect_env_cli'
 
 [tool.setuptools.packages.find]
 include = [
diff --git a/todd/base/__init__.py b/todd/base/__init__.py
index d68fe67..2f66838 100644
--- a/todd/base/__init__.py
+++ b/todd/base/__init__.py
@@ -3,6 +3,7 @@
 from . import patches
 from .bboxes import *
 from .configs import *
+from .envs import *
 from .eta import *
 from .filters import *
 from .logger import *
diff --git a/todd/utils/envs.py b/todd/base/envs.py
similarity index 73%
rename from todd/utils/envs.py
rename to todd/base/envs.py
index 7360366..1d53439 100644
--- a/todd/utils/envs.py
+++ b/todd/base/envs.py
@@ -11,18 +11,25 @@ __all__ = [
     'cuda_home',
     'git_commit_id',
     'git_status',
+    'collect_env',
 ]
 
+import argparse
 import importlib.util
 import

### Optimize

In [17]:
config = todd.Config(
    type="IterBasedTrainer",
    name="optimize_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

[2024-02-07 15:04:32,067 56800:140704380690048][base.py:55 todd.IterBasedTrainer.optimize_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-07 15:04:32,071 56800:140704380690048][log.py:91 todd.IterBasedTrainer.optimize_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([3, 4]), 'y': tensor([6, 8])} weight=0.000 loss=7.000
[2024-02-07 15:04:32,074 56800:140704380690048][log.py:91 todd.IterBasedTrainer.optimize_callback after_run_iter] INFO: Iter [2/8] batch={'x': tensor([ 2, 10]), 'y': tensor([ 4, 20])} weight=0.018 loss=11.895
[2024-02-07 15:04:32,076 56800:140704380690048][log.py:91 todd.IterBasedTrainer.optimize_callback after_run_iter] INFO: Iter [3/8] batch={'x': tensor([8, 6]), 'y': tensor([16, 12])} weight=0.047 loss=13.667
[2024-02-07 15:04:32,079 56800:140704380690048][log.py:91 todd.IterBasedTrainer.optimize_callback after_run_iter] INFO: Iter [4/8] batch={'x': tensor([5, 7]), 'y': tensor([10, 14])} weight=0.082 loss=11.505
[2024-02-07 

### Learning Rate Schedule

In [18]:
config = todd.Config(
    type="IterBasedTrainer",
    name="lr_schedule_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type="OptimizeCallback"),
        dict(
            type="LRScheduleCallback",
            lr_scheduler=dict(type="LinearLR", total_iters=5),
        ),
        dict(type="LogCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-07 15:04:32,100 56800:140704380690048][base.py:55 todd.IterBasedTrainer.lr_schedule_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-07 15:04:32,102 56800:140704380690048][log.py:91 todd.IterBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([9, 5]), 'y': tensor([18, 10])} weight=0.000 loss=14.000 lr=['1.667e-03']
[2024-02-07 15:04:32,104 56800:140704380690048][log.py:91 todd.IterBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [2/8] batch={'x': tensor([ 2, 10]), 'y': tensor([ 4, 20])} weight=0.012 loss=11.930 lr=['2.333e-03']
[2024-02-07 15:04:32,106 56800:140704380690048][log.py:91 todd.IterBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [3/8] batch={'x': tensor([3, 7]), 'y': tensor([ 6, 14])} weight=0.026 loss=9.872 lr=['3.000e-03']
[2024-02-07 15:04:32,108 56800:140704380690048][log.py:91 todd.IterBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [4/8] batch={'x': tensor([

In [19]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="lr_schedule_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=4),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type="OptimizeCallback"),
        dict(
            type="LRScheduleCallback",
            lr_scheduler=dict(type="LinearLR", total_iters=3),
            by_epoch=True,
        ),
        dict(type="LogCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=5,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-07 15:04:32,129 56800:140704380690048][base.py:55 todd.EpochBasedTrainer.lr_schedule_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-07 15:04:32,130 56800:140704380690048][log.py:97 todd.EpochBasedTrainer.lr_schedule_callback before_run_epoch] INFO: Epoch [1/5]
[2024-02-07 15:04:32,133 56800:140704380690048][log.py:91 todd.EpochBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [1/10] batch={'x': tensor([2, 1]), 'y': tensor([4, 2])} weight=0.000 loss=3.000 lr=['1.667e-03']
[2024-02-07 15:04:32,135 56800:140704380690048][log.py:91 todd.EpochBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [2/10] batch={'x': tensor([4, 3]), 'y': tensor([8, 6])} weight=0.002 loss=6.991 lr=['1.667e-03']
[2024-02-07 15:04:32,136 56800:140704380690048][log.py:97 todd.EpochBasedTrainer.lr_schedule_callback before_run_epoch] INFO: Epoch [2/5]
[2024-02-07 15:04:32,138 56800:140704380690048][log.py:91 todd.EpochBasedTrainer.lr_schedule_callback after

### Learning Rate Scale

In [20]:
config = todd.Config(
    type="IterBasedTrainer",
    name="lr_scale_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type="OptimizeCallback"),
        dict(
            type="LRScaleCallback",
            lr_scaler=dict(base_batch_size=1),
        ),
        dict(type="LogCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-07 15:04:32,169 56800:140704380690048][lr.py:93 todd.IterBasedTrainer.lr_scale_callback _scale_lr] INFO: base_batch_size=1 batch_size=2 lr_scaler=2.000
[2024-02-07 15:04:32,170 56800:140704380690048][base.py:55 todd.IterBasedTrainer.lr_scale_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-07 15:04:32,173 56800:140704380690048][log.py:91 todd.IterBasedTrainer.lr_scale_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([5, 7]), 'y': tensor([10, 14])} weight=0.000 loss=12.000
[2024-02-07 15:04:32,175 56800:140704380690048][log.py:91 todd.IterBasedTrainer.lr_scale_callback after_run_iter] INFO: Iter [2/8] batch={'x': tensor([10,  2]), 'y': tensor([20,  4])} weight=0.060 loss=11.640
[2024-02-07 15:04:32,177 56800:140704380690048][log.py:91 todd.IterBasedTrainer.lr_scale_callback after_run_iter] INFO: Iter [3/8] batch={'x': tensor([4, 6]), 'y': tensor([ 8, 12])} weight=0.120 loss=9.400
[2024-02-07 15:04:32,178 56800:140704380690048][log.py:9

### Checkpoint

In [21]:
config = todd.Config(
    type="IterBasedTrainer",
    name="checkpoint_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    iter_5 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'iter_5'
    for f in iter_5.glob('*.pth'):
        print(f'{f.name}:')
        pprint(torch.load(f, 'cpu'))
        print()

    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(iter_5),
        )
    runner.run()


[2024-02-07 15:04:32,209 56800:140704380690048][base.py:55 todd.IterBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-07 15:04:32,211 56800:140704380690048][log.py:91 todd.IterBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([10,  3]), 'y': tensor([20,  6])} weight=0.000 loss=13.000
[2024-02-07 15:04:32,212 56800:140704380690048][checkpoint.py:80 todd.IterBasedTrainer.checkpoint_callback _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpqfpwprlf/checkpoint_callback/checkpoints/iter_1
[2024-02-07 15:04:32,217 56800:140704380690048][log.py:91 todd.IterBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [2/8] batch={'x': tensor([6, 9]), 'y': tensor([12, 18])} weight=0.032 loss=14.756
[2024-02-07 15:04:32,218 56800:140704380690048][checkpoint.py:80 todd.IterBasedTrainer.checkpoint_callback _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpqfpwprlf
└── checkpoint_callback
    └── checkpoints
        ├── iter_1
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_2
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_3
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_4
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_5
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_6
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── opt

[2024-02-07 15:04:32,686 56800:140704380690048][checkpoint.py:54 todd.IterBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpqfpwprlf/checkpoint_callback/checkpoints/iter_5
[2024-02-07 15:04:32,690 56800:140704380690048][base.py:82 todd.IterBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-02-07 15:04:32,691 56800:140704380690048][base.py:55 todd.IterBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-07 15:04:32,694 56800:140704380690048][log.py:91 todd.IterBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [6/8] batch={'x': tensor([8, 5]), 'y': tensor([16, 10])} weight=0.137 loss=12.106
[2024-02-07 15:04:32,695 56800:140704380690048][checkpoint.py:80 todd.IterBasedTrainer.checkpoint_callback _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpqfpwprlf/checkpoint_callback/checkpoints/

strategy.pth:
{}

optim.pth:
{'param_groups': [{'dampening': 0,
                   'differentiable': False,
                   'foreach': None,
                   'lr': 0.005,
                   'maximize': False,
                   'momentum': 0,
                   'nesterov': False,
                   'params': [0],
                   'weight_decay': 0}],
 'state': {0: {'momentum_buffer': None}}}

meta.pth:
{'iter_': 5}

model.pth:
OrderedDict([('_weight', tensor(0.1375))])

callbacks.pth:
{'callbacks': [{}, {}, {}]}



In [22]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="checkpoint_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=2),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    iter_8 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'iter_8'
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(iter_8),
        )
    runner.run()

    !echo
    !echo {'-' * 20}
    !echo

    iter_10 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'iter_10'
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(iter_10),
        )
    runner.run()


[2024-02-07 15:04:32,747 56800:140704380690048][base.py:55 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-07 15:04:32,748 56800:140704380690048][log.py:97 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [1/3]
[2024-02-07 15:04:32,750 56800:140704380690048][log.py:91 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [1/15] batch={'x': tensor([5, 1]), 'y': tensor([10,  2])} weight=0.000 loss=6.000
[2024-02-07 15:04:32,753 56800:140704380690048][log.py:91 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [2/15] batch={'x': tensor([9, 4]), 'y': tensor([18,  8])} weight=0.015 loss=12.903
[2024-02-07 15:04:32,770 56800:140704380690048][checkpoint.py:80 todd.EpochBasedTrainer.checkpoint_callback _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpp2giquc4/checkpoint_callback/checkpoints/iter_2
[2024-02-07 15:04:32,774 56800:14070


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpp2giquc4
└── checkpoint_callback
    └── checkpoints
        ├── iter_10
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_12
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_14
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_2
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_4
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_6
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── 

[2024-02-07 15:04:33,245 56800:140704380690048][checkpoint.py:54 todd.EpochBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpp2giquc4/checkpoint_callback/checkpoints/iter_8
[2024-02-07 15:04:33,249 56800:140704380690048][base.py:82 todd.EpochBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-02-07 15:04:33,250 56800:140704380690048][base.py:55 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-07 15:04:33,251 56800:140704380690048][log.py:97 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [2/3]
[2024-02-07 15:04:33,255 56800:140704380690048][log.py:91 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [9/15] batch={'x': tensor([4, 5]), 'y': tensor([ 8, 10])} weight=0.245 loss=7.898
[2024-02-07 15:04:33,257 56800:140704380690048][log.py:91 todd.EpochBasedTrainer.checkpoint_c


--------------------



[2024-02-07 15:04:33,743 56800:140704380690048][checkpoint.py:54 todd.EpochBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpp2giquc4/checkpoint_callback/checkpoints/iter_10
[2024-02-07 15:04:33,747 56800:140704380690048][base.py:82 todd.EpochBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-02-07 15:04:33,748 56800:140704380690048][base.py:55 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-07 15:04:33,749 56800:140704380690048][log.py:97 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [3/3]
[2024-02-07 15:04:33,752 56800:140704380690048][log.py:91 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [11/15] batch={'x': tensor([9, 4]), 'y': tensor([18,  8])} weight=0.293 loss=11.099
[2024-02-07 15:04:33,754 56800:140704380690048][log.py:91 todd.EpochBasedTrainer.checkpoin

In [23]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="checkpoint_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=1, by_epoch=True),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    epoch_2 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'epoch_2'
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(epoch_2),
        )
    runner.run()


[2024-02-07 15:04:33,796 56800:140704380690048][base.py:55 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-07 15:04:33,797 56800:140704380690048][log.py:97 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [1/3]
[2024-02-07 15:04:33,799 56800:140704380690048][log.py:91 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [1/15] batch={'x': tensor([3, 1]), 'y': tensor([6, 2])} weight=0.000 loss=4.000
[2024-02-07 15:04:33,801 56800:140704380690048][log.py:91 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [2/15] batch={'x': tensor([10,  5]), 'y': tensor([20, 10])} weight=0.010 loss=14.925
[2024-02-07 15:04:33,803 56800:140704380690048][log.py:91 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [3/15] batch={'x': tensor([9, 4]), 'y': tensor([18,  8])} weight=0.047 loss=12.691
[2024-02-07 15:04:33,805 56800:140704380690048][log.py:91 todd.Epoc


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpbnsfxij1
└── checkpoint_callback
    └── checkpoints
        ├── epoch_1
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── epoch_2
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── epoch_3
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        └── latest -> epoch_3

7 directories, 15 files



[2024-02-07 15:04:34,273 56800:140704380690048][checkpoint.py:54 todd.EpochBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpbnsfxij1/checkpoint_callback/checkpoints/epoch_2
[2024-02-07 15:04:34,276 56800:140704380690048][base.py:82 todd.EpochBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-02-07 15:04:34,277 56800:140704380690048][base.py:55 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-07 15:04:34,278 56800:140704380690048][log.py:97 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [3/3]
[2024-02-07 15:04:34,282 56800:140704380690048][log.py:91 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [11/15] batch={'x': tensor([8, 7]), 'y': tensor([16, 14])} weight=0.275 loss=12.938
[2024-02-07 15:04:34,284 56800:140704380690048][log.py:91 todd.EpochBasedTrainer.checkpoin

### Monitor

In [24]:
class CustomError(RuntimeError):
    pass

In [25]:
@todd.RunnerRegistry.register_()
class FaultyValidator(todd.runners.Validator):

    def _run_iter(self, *args, **kwargs) -> NoReturn:
        raise CustomError("faulty runner")

In [26]:
config = todd.Config(
    type='FaultyValidator',
    name='monitor_callback',
    dataloader=dict(batch_size=1, dataset=dict(type='RunnerDataset', n=20)),
    strategy=dict(type='BaseStrategy', model=dict(type='RunnerModel')),
    callbacks=[
        dict(type='MonitorCallback'),
        dict(type='LogCallback', interval=5, with_file_handler=True),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    try:
        runner.run()
    except CustomError as e:
        pass

    !echo
    !cat {work_dirs}/monitor_callback/*.log


[2024-02-07 15:04:34,330 56800:140704380690048][base.py:55 todd.FaultyValidator.monitor_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-07 15:04:34,332 56800:140704380690048][monitor.py:26 todd.FaultyValidator.monitor_callback __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader._SingleProcessDataLoaderIter object at 0x1575fcb90>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/base.py", line 223, in _run
    memo = self._run_iter(batch, memo)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/ipykernel_56800/1715875531.py", line 5, in _run_iter
    raise CustomError("faulty runner")
CustomError: faulty runner



[2024-02-07 15:04:34,330 56800:140704380690048][base.py:55 todd.FaultyValidator.monitor_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-07 15:04:34,332 56800:140704380690048][monitor.py:26 todd.FaultyValidator.monitor_callback __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader._SingleProcessDataLoaderIter object at 0x1575fcb90>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/base.py", line 223, in _run
    memo = self._run_iter(batch, memo)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/ipykernel_56800/1715875531.py", line 5, in _run_iter
    raise CustomError("faulty runner")
CustomError: faulty runner


### Priorities

## Strategies

In [27]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="strategy_load_model_from",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=1, by_epoch=True),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !echo {'-' * 20}
    !echo

    epoch_2 = (pathlib.Path(work_dirs) / 'strategy_load_model_from' / 'checkpoints' / 'epoch_2' / 'model.pth')
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.strategy.load_model_from(epoch_2)
    runner.run()


[2024-02-07 15:04:34,648 56800:140704380690048][base.py:55 todd.EpochBasedTrainer.strategy_load_model_from __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-07 15:04:34,650 56800:140704380690048][log.py:97 todd.EpochBasedTrainer.strategy_load_model_from before_run_epoch] INFO: Epoch [1/3]
[2024-02-07 15:04:34,653 56800:140704380690048][log.py:91 todd.EpochBasedTrainer.strategy_load_model_from after_run_iter] INFO: Iter [1/15] batch={'x': tensor([5, 4]), 'y': tensor([10,  8])} weight=0.000 loss=9.000
[2024-02-07 15:04:34,656 56800:140704380690048][log.py:91 todd.EpochBasedTrainer.strategy_load_model_from after_run_iter] INFO: Iter [2/15] batch={'x': tensor([1, 2]), 'y': tensor([2, 4])} weight=0.022 loss=2.966
[2024-02-07 15:04:34,658 56800:140704380690048][log.py:91 todd.EpochBasedTrainer.strategy_load_model_from after_run_iter] INFO: Iter [3/15] batch={'x': tensor([8, 3]), 'y': tensor([16,  6])} weight=0.030 loss=10.835
[2024-02-07 15:04:34,660 56800:14070438069004


--------------------



[2024-02-07 15:04:35,115 56800:140704380690048][base.py:55 todd.EpochBasedTrainer.strategy_load_model_from __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-07 15:04:35,116 56800:140704380690048][base.py:97 todd.EpochBasedTrainer.strategy_load_model_from load_model_from] INFO: Loading model from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpo8hstrjt/strategy_load_model_from/checkpoints/epoch_2/model.pth
[2024-02-07 15:04:35,119 56800:140704380690048][base.py:82 todd.EpochBasedTrainer.strategy_load_model_from load_model_state_dict] INFO: <All keys matched successfully>
[2024-02-07 15:04:35,120 56800:140704380690048][log.py:97 todd.EpochBasedTrainer.strategy_load_model_from before_run_epoch] INFO: Epoch [1/3]
[2024-02-07 15:04:35,123 56800:140704380690048][log.py:91 todd.EpochBasedTrainer.strategy_load_model_from after_run_iter] INFO: Iter [1/15] batch={'x': tensor([9, 2]), 'y': tensor([18,  4])} weight=0.275 loss=9.488
[2024-02-07 15:04:35,125 56800:1407043806

## Dry Run

In [28]:
todd.Store.DRY_RUN = True